In [109]:
%matplotlib inline
from __future__ import division, print_function
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import ZeroPadding2D
from keras.layers import MaxPooling2D, \
    Lambda, \
    Dense, \
    Dropout, \
    Flatten
import numpy as np
from keras.preprocessing import image 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from utils import plots

In [47]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

In [89]:
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1]    

In [56]:
idg = image.ImageDataGenerator()
train_generator = idg.flow_from_directory('/mnt/data/dogscats/train/', batch_size=4)

Found 23000 images belonging to 2 classes.


In [112]:
def addConvBlock(model, layers, filters):
    for layer in range(0, layers):
        model.add(ZeroPadding2D(padding=(1, 1)))
        model.add(Convolution2D(nb_filter=filters, nb_row=3, nb_col=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

In [113]:
def addFCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [114]:
def create():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3, 224, 224), output_shape=(3,224,224)))
    addConvBlock(model, 2, 64)
    addConvBlock(model, 2, 128)
    addConvBlock(model, 3, 256)
    addConvBlock(model, 3, 512)
    addConvBlock(model, 3, 512)
    model.add(Flatten())
    addFCBlock(model)
    addFCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    return model

In [115]:
model = create()

In [116]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_6 (Lambda)                (None, 3, 224, 224)   0           lambda_input_6[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_6[0][0]                   
____________________________________________________________________________________________________
convolution2d_30 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_30[0][0]           
___________________________________________________________________________________________

In [93]:
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(nb_filter=64, nb_row=3, nb_col=3, activation='relu'))

In [94]:
model.add(MaxPooling2D((2, 2), strides=(2,2)))

In [ ]:
def ConvBlock(model, layers, filters):
    for i in range(0, layers):
        model.add(ZeroPadding2D((1, 1))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))

In [41]:
idg = image.ImageDataGenerator()
train_generator = idg.flow_from_directory('/mnt/data/dogscats/train/', batch_size=4)

Found 23000 images belonging to 2 classes.


In [42]:
abc = next(train_generator)

In [44]:
abc[0].shape

(4, 3, 256, 256)